In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib
plt.style.use('figures.mplstyle')
import leptoninjector as LI
from MuonSimulationHelpers import *

In [ ]:
Erange = np.logspace(2,10,100)
for k in DIS_xs:
    if 'bar' in k:
        xs = np.array([DIS_xs[k].TotalCrossSection(numubar,E,target_type) for E in Erange])
        plt.plot(Erange,xs,label=r'$\overline{\nu}$' + k[-2:],ls='--')
#         for primary,tag in zip([nuebar,numubar,nutaubar],
#                                [r'$\overline{\nu}_e$',r'$\overline{\nu}_\mu$',r'$\overline{\nu}_\tau$']):
            
#             xs = np.array([DIS_xs[k].TotalCrossSection(primary,E,target_type) for E in Erange])
#             plt.plot(Erange,xs,label=tag + k[-2:],ls='--')
    else:
        xs = np.array([DIS_xs[k].TotalCrossSection(numu,E,target_type) for E in Erange])
        plt.plot(Erange,xs,label=r'$\nu$' + k[-2:])
#         for primary,tag in zip([nue,numu,nutau],
#                                [r'$\nu_e$',r'${\nu}_\mu$',r'${\nu}_\tau$']):
            
#             xs = np.array([DIS_xs[k].TotalCrossSection(primary,E,target_type) for E in Erange])
#             plt.plot(Erange,xs,label=tag + k[-2:])
plt.loglog()
plt.legend()
plt.xlabel('Energy [GeV')
plt.ylabel('Cross Section [cm^2]')
plt.show()

In [ ]:
N = 1000
light_simulation = MuonSimulation('Data/MuonSimulation/Large_EPOSLHC_light.parquet')
# light_simulation.SampleSecondaryMomenta(N)
# light_simulation.DumpData('Data/MuonSimulation/Large_EPOSLHC_light.parquet')
charm_simulation = MuonSimulation('Data/MuonSimulation/Large_BKRS_charm.parquet')
# charm_simulation.SampleSecondaryMomenta(N)
# charm_simulation.DumpData('Data/MuonSimulation/Large_BKRS_charm.parquet')


In [ ]:
kwargs = {'bins':np.logspace(-1,4,100),
          'histtype':'step',
          'weights':light_simulation.data['wgt']*150*1000}

momenta = np.array([light_simulation.data['E'],
                   light_simulation.data['px'],
                   light_simulation.data['py'],
                   light_simulation.data['pz']]).T
muon_momenta = np.array([light_simulation.data['E_lep'],
                   light_simulation.data['px_lep'],
                   light_simulation.data['py_lep'],
                   light_simulation.data['pz_lep']]).T
target_momenta = np.array([light_simulation.data['E_had'],
                   light_simulation.data['px_had'],
                   light_simulation.data['py_had'],
                   light_simulation.data['pz_had']]).T
plt.hist(momenta[:,0],color='r',label='neutrino',**kwargs)
wnorm = sum(kwargs['weights'])/sum(light_simulation.data.query('E_lep>0')['wgt']*150*1000)
kwargs['weights']*=wnorm

plt.hist(muon_momenta[:,0],color='b',label='lepton',**kwargs)
plt.hist(target_momenta[:,0],color='g',label='target',**kwargs)

kwargs['ls'] = '--'
kwargs['weights'] = charm_simulation.data['wgt']*150*1000

momenta = np.array([charm_simulation.data['E'],
                   charm_simulation.data['px'],
                   charm_simulation.data['py'],
                   charm_simulation.data['pz']]).T
muon_momenta = np.array([charm_simulation.data['E_lep'],
                   charm_simulation.data['px_lep'],
                   charm_simulation.data['py_lep'],
                   charm_simulation.data['pz_lep']]).T
target_momenta = np.array([charm_simulation.data['E_had'],
                   charm_simulation.data['px_had'],
                   charm_simulation.data['py_had'],
                   charm_simulation.data['pz_had']]).T
plt.hist(momenta[:,0],color='r',**kwargs)
wnorm = sum(kwargs['weights'])/sum(charm_simulation.data.query('E_lep>0')['wgt']*150*1000)
kwargs['weights']*=wnorm
plt.hist(muon_momenta[:,0],color='b',**kwargs)
plt.hist(target_momenta[:,0],color='g',**kwargs)

plt.plot([],[],color='black',label=r'$\pi,K$')
plt.plot([],[],color='black',ls='--',label=r'$D,\Lambda_c$')

plt.xlabel('Energy [GeV]')
plt.ylabel(r'Number of Neutrinos / bin / 150 fb$^{-1}$')
plt.loglog()
plt.legend()
plt.show()

In [ ]:
light_simulation.CalculateLakeIntersectionsFromIP('LHCb',N)

In [ ]:
int_distances = {}
for ints in light_simulation.lake_intersections:
    x0 = np.array(LHC_data.loc['LHCb',['X','Y','Z']])
    for i,x in enumerate(ints):
        if i not in int_distances:
            int_distances[i] = []
        int_distances[i].append(np.linalg.norm(x-x0))
        x0 = x
        
for k,v in int_distances.items():
    plt.hist(v,bins=np.linspace(0,30000,50),alpha=0.5)    

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.fill(list(Lake_data['Longitude']) + [Lake_data['Longitude'][0]],
         list(Lake_data['Latitude']) + [Lake_data['Latitude'][0]],
         color='blue',label='Lake Geneva',alpha=0.2,linewidth=2)
Lake_Crossings = {}
for k,color in zip(list(LHC_data.index),mcolors.TABLEAU_COLORS):
    Lake_Crossings[k] = []
    lines = plot_tangent_line_lat_long(LHC,
                                       np.array(LHC_data.loc[k,['X','Y','Z']]),
                                       np.array(LHC_data.loc[k,['CrossingOrientation','CrossingAngle']]),
                                       limit=1 if 'Point' in k else 60000)
    center, earth_points = lines[0] # all lines are essentially the same
    plt.plot([p[1] for p in earth_points],
            [p[0] for p in earth_points],color=color,ls='--')
    plt.scatter(center[1],
                center[0],
                marker='*',color=color,label=k)
    if 'Point' in k: continue # don't consider points without collisions
    int1,int2 = calculate_intersections_with_lake(LHC,
                                                np.array(LHC_data.loc[k,['X','Y','Z']]),
                                                np.array(LHC_data.loc[k,['CrossingOrientation','CrossingAngle']]),
                                                np.array(Lake_data[['Latitude','Longitude']]),
                                                limit=50000000)
    for intersections in (int1,int2):
        if(type(intersections) == shapely.geometry.MultiLineString):
            for intersection in intersections.geoms:
                for coord in list(intersection.coords):
                    Lake_Crossings[k].append(coord)
                    plt.scatter(coord[1],coord[0],color=color)
        else:
            for coord in list(intersections.coords):
                Lake_Crossings[k].append(coord)
                plt.scatter(coord[1],coord[0],color=color)
for ints in light_simulation.lake_intersections_lat_long:
    if len(ints)>0:
        plt.plot(np.array(ints)[:,1],np.array(ints)[:,0],color='black',alpha=0.005)
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.legend(loc='lower right')
plt.xlim(5.8,7.0)
plt.ylim(46.1,46.6)
plt.hist2d(light_simulation.surface_intersections_lat_long[:,1],
           light_simulation.surface_intersections_lat_long[:,0],
           bins=(np.linspace(5.5,7.0,50),np.linspace(45.9,46.6,50)),
           weights = (light_simulation.data['wgt']*150*1000)[:1000],
           norm=matplotlib.colors.LogNorm())
plt.show()


In [ ]:
light_simulation.CalculateSurfaceIntersectionFromIP('LHCb',N)

In [ ]:
plt.hist2d(light_simulation.surface_intersections_lat_long[:,1],
           light_simulation.surface_intersections_lat_long[:,0],
           bins=(np.linspace(5.5,7.0,50),np.linspace(45.9,46.6,50)),
           weights = (light_simulation.data['wgt']*150*1000)[:1000],
           norm=matplotlib.colors.LogNorm())
plt.colorbar()
plt.show()